# fetch data

In [1]:
import requests
import csv
from datetime import datetime, timedelta

api_url = "https://neptune.kyogojo.com/api/statistics/get-multiple?stations=BSLG-003&days=1743465600000,1743552000000,1743638400000,1743724800000,1743811200000,1743897600000,1743984000000,1744070400000,1744156800000,1744243200000,1744329600000,1744416000000,1744502400000,1744588800000,1744675200000,1744761600000,1744848000000,1744934400000"

response = requests.get(api_url)
data = response.json()

pressure_data = data["payload"]["data"][0]["pressure"]

csv_filename = r"C:\Users\Sarah\Desktop\trend_analysis\data\water_pressureBSLG003.csv"

# save to CSV
with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["value", "time"])

    for entry in pressure_data:
        value = entry["value"]
        timestamp = entry["time"]
        
        # convert UTC time to Philippine Standard Time (UTC+8)
        utc_time = datetime.utcfromtimestamp(timestamp / 1000)  #milliseconds to seconds
        pst_time = utc_time + timedelta(hours=8)  

        formatted_time = pst_time.strftime("%Y-%m-%d %H:%M:%S")    
        writer.writerow([value, formatted_time])

print(f"Data saved to {csv_filename}")


C:\Users\Sarah\AppData\Local\Temp\ipykernel_3692\1144362513.py:24: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  utc_time = datetime.utcfromtimestamp(timestamp / 1000)  #milliseconds to seconds


Data saved to C:\Users\Sarah\Desktop\trend_analysis\data\water_pressureBSLG003.csv


#  data

In [1]:
#importing required functiond from sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-003-JAN-MARCH.csv"
df = pd.read_csv(file_path)
df.head()
# Define the ranges
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)
range_counts = df['Value Range'].value_counts()

print(range_counts)


Value Range
30-39    88414
20-29    67463
11-19     8706
40-49     4969
0-10       148
50-59      110
60-69        0
70-79        0
80-89        0
Name: count, dtype: int64


In [5]:
file_path = r"C:\Users\Sarah\Desktop\trend_analysis\data\BSLG-003-JAN-MARCH.csv"
df = pd.read_csv(file_path)

# Define the condition
condition1 = (df['value'] >= 30) & (df['value'] <= 39)

# Drop the specified number of rows based on the condition
rows_to_drop_condition1 = df[condition1].index[:60000]
df = df.drop(rows_to_drop_condition1)

# Define ranges and corresponding labels
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]
labels = ['0-10', '11-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80-89']

# Categorize the values into defined ranges
df['Value Range'] = pd.cut(df['value'], bins=bins, labels=labels, right=False)
range_counts = df['Value Range'].value_counts()

# Display the results
print("Rows dropped successfully!")
print(range_counts)


Rows dropped successfully!
Value Range
20-29    67463
30-39    28414
11-19     8706
40-49     4969
0-10       148
50-59      110
60-69        0
70-79        0
80-89        0
Name: count, dtype: int64


In [6]:

print(df.shape)


(109810, 3)


In [7]:
# column to datetime
df['time'] = pd.to_datetime(df['time'])

# extract time features
df['year'] = df['time'].dt.year
df['month'] = df['time'].dt.month
df['day'] = df['time'].dt.day
df['hour'] = df['time'].dt.hour
df['minute'] = df['time'].dt.minute
df['second'] = df['time'].dt.second
df['millisecond'] = df['time'].dt.microsecond // 1000  # microseconds to milliseconds

df.drop(columns=['time'], inplace=True)
df.drop(columns=['Value Range'], inplace=True)

# Features (X), target variable (y)
X = df.drop(columns=['value'])
y = df['value']

df.head()

,value,year,month,day,hour,minute,second,millisecond
0,39.34,2025,1,1,0,0,42,0
1,40.18,2025,1,1,0,1,42,0
2,40.87,2025,1,1,0,2,41,0
3,41.18,2025,1,1,0,3,41,0
4,41.02,2025,1,1,0,4,41,0


In [8]:
# Ensure data is sorted before training
df = df.sort_values(by=['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond']).reset_index(drop=True)
df.head()


,value,year,month,day,hour,minute,second,millisecond
0,39.34,2025,1,1,0,0,42,0
1,40.18,2025,1,1,0,1,42,0
2,40.87,2025,1,1,0,2,41,0
3,41.18,2025,1,1,0,3,41,0
4,41.02,2025,1,1,0,4,41,0


In [9]:
# missing values
missing_values = df.isnull().sum()
print("Missing Values per Column:")
print(missing_values)

Missing Values per Column:
value          0
year           0
month          0
day            0
hour           0
minute         0
second         0
millisecond    0
dtype: int64


In [10]:
# 85% train, 15% test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
# standardize features (x)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [12]:
import joblib

# Save the scaler
joblib.dump(scaler, r"C:\Users\Sarah\Desktop\trend_analysis\model\scaler_003.pkl")
print("Scaler saved successfully!")


Scaler saved successfully!


In [13]:
# Wrap the scaled arrays with the original feature names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [14]:
# Initializing base models
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
lr_model = LinearRegression()

# Creating the stacked model
stacked_model = StackingRegressor(estimators=[('rf', rf_model), ('lr', lr_model)])

# train
stacked_model.fit(X_train_scaled, y_train)


StackingRegressor(estimators=[('rf', RandomForestRegressor(random_state=42)),
                              ('lr', LinearRegression())])

In [15]:
accuracy = stacked_model.score(X_test_scaled, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.9873838154809464


Accuracy: 0.9897540714474308


In [30]:
# train
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,
                              random_state=42)
model.fit(X_train_scaled, y_train)
print(model)


RandomForestRegressor(random_state=42)


In [16]:


from sklearn.model_selection import cross_val_score
# R-squared score
train_score = stacked_model.score(X_train_scaled, y_train)
test_score = stacked_model.score(X_test_scaled, y_test)

# Cross-validation
cv_scores = cross_val_score(stacked_model, X_train_scaled, y_train, cv=5)

print(f"Train R-squared: {train_score:.4f}")
print(f"Test R-squared: {test_score:.4f}")
print(f"Cross-validation score: {cv_scores.mean():.4f}")


Train R-squared: 0.9982
Test R-squared: 0.9874
Cross-validation score: 0.9832


In [31]:
from sklearn.model_selection import cross_val_score
# R-squared score
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)

# Cross-validation
cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)

print(f"Train R-squared: {train_score:.4f}")
print(f"Test R-squared: {test_score:.4f}")
print(f"Cross-validation score: {cv_scores.mean():.4f}")


Train R-squared: 0.9982
Test R-squared: 0.9876
Cross-validation score: 0.9818


In [32]:
# Predict on test set
y_pred = model.predict(X_test_scaled)

# Show some predictions
df_predictions = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_pred})
print(df_predictions.tail(10))


       Actual  Predicted
28242   35.68    35.5897
28243   34.46    34.4134
28244   38.08    38.1871
28245   26.05    26.2658
28246   16.78    16.6994
28247   30.28    30.2946
28248   30.55    30.6132
28249   30.01    30.2227
28250   39.27    38.9724
28251   26.82    27.0408


In [17]:
import joblib

# Save the trained model
joblib.dump(stacked_model, r"C:\Users\Sarah\Desktop\trend_analysis\model\random_forest_bslg_003.pkl")

print("Model saved successfully!")


Model saved successfully!


In [18]:
# Load the model
loaded_model = joblib.load(r"C:\Users\Sarah\Desktop\trend_analysis\model\random_forest_model.pkl")

print("Model loaded successfully!")

# Now you can make predictions using the loaded model
y_pred = loaded_model.predict(X_test)


Model loaded successfully!


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta

# Convert X_test back to DataFrame with column names
X_test_df = pd.DataFrame(X_test, columns=['year', 'month', 'day', 'hour', 'minute', 'second', 'millisecond'])
X_test_df['Actual'] = y_test.values
X_test_df['Predicted'] = y_pred

# Group by day (calculate average actual and predicted value per day)
daily_avg = X_test_df.groupby(['year', 'month', 'day'])[['Actual', 'Predicted']].mean().reset_index()

# Create date column for plotting
daily_avg['date'] = pd.to_datetime(daily_avg[['year', 'month', 'day']])

### 📌 Generate Future Dates for Prediction
future_days = 7  # Predict next 7 days
last_date = daily_avg['date'].max()
future_dates = [last_date + timedelta(days=i) for i in range(1, future_days + 1)]

# Create future DataFrame
future_df = pd.DataFrame({
    'year': [d.year for d in future_dates],
    'month': [d.month for d in future_dates],
    'day': [d.day for d in future_dates],
    'hour': [0] * future_days,  # Assuming midnight for simplicity
    'minute': [0] * future_days,
    'second': [0] * future_days,
    'millisecond': [0] * future_days
})

# Standardize future data
future_scaled = scaler.transform(future_df)

# Predict future water pressure
future_df['Predicted'] = model.predict(future_scaled)
future_df['date'] = future_dates  # Assign future date column

# Append future predictions to daily_avg
extended_df = pd.concat([daily_avg, future_df], ignore_index=True)

### 📌 Plot Actual vs. Predicted (with Future Predictions)
plt.figure(figsize=(12, 6))
sns.lineplot(data=extended_df, x='date', y='Actual', label='Actual', marker='o', linewidth=2, color='blue')
sns.lineplot(data=extended_df, x='date', y='Predicted', label='Predicted (Including Future)', marker='s', linewidth=2, linestyle='dashed', color='red')

# Format the plot
plt.xlabel("Date")
plt.ylabel("Water Pressure")
plt.title("Actual vs. Predicted Water Pressure Trends (with Future Predictions)")
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Show the plot
plt.show()
